In [1]:
%load_ext autoreload
%autoreload 2

https://www.stata.com/features/overview/logistic-regression/

In [111]:
import numpy as np
import pandas as pd

In [112]:
import quantmetrics
from quantmetrics.datasets import webuse
from quantmetrics.discrete_choice import logistic

In [113]:
webuse('lbw')

In [114]:
data = pd.read_stata("data/lbw")

In [115]:
X = data.drop('low', axis=1)
y = data['low']

In [116]:
X_ = X.copy()
bwt = X['bwt']
X_['bwt'] = (bwt-bwt.mean()) / bwt.std()
lwt = X['lwt']
X_['lwt'] = (lwt-lwt.mean()) / lwt.std()
age = X['age']
X_['age'] = (age-age.mean()) / age.std()
X_.drop(columns=['id', 'bwt', 'ftv'], inplace=True)

In [117]:
X_.head(1)

,age,lwt,race,smoke,ptl,ht,ui
0,-0.79984,1.706611,black,nonsmoker,0,0,1


In [118]:
init_value = np.array([1, 1.1, 1, 1, 1, 1, 1.2, 1, 0.5])

In [127]:
logistic(X_, y, init_value)

        age       lwt  ptl  ht  ui  race_black  race_other  smoke_smoker  \
0 -0.799840  1.706611    0   0   1           1           0             0   
1  1.842328  0.823541    0   0   0           0           1             0   
2 -0.611114 -0.811774    0   0   0           0           0             1   
3 -0.422387 -0.713655    0   0   1           0           0             1   
4 -0.988566 -0.746361    0   0   1           0           0             1   

   _cons  
0      1  
1      1  
2      1  
3      1  
4      1  
Logistic regression
Number of obs = 189
Log likelihood = -100.724
LR chi2(42.195) = 42.195
Prob > chi2 = 0.42195
Pseudo R2 = 0.42195
Odds Ratio = {'age': 42.195, 'lwt': 42.195}
Std. Err. = {'age': 42.195, 'lwt': 42.195}
z = {'age': 42.195, 'lwt': 42.195}
p_z = {'age': 42.195, 'lwt': 42.195}
95% Conf. Interval bottom = {'age': 42.195, 'lwt': 42.195}
95% Conf. Interval top = {'age': 42.195, 'lwt': 42.195}
      fun: 100.7239955662642
 hess_inv: array([[ 0.03886682, -0.004710

In [8]:
data.head()

,id,low,age,lwt,race,smoke,ptl,ht,ui,ftv,bwt
0,85,0,19,182,black,nonsmoker,0,0,1,0,2523
1,86,0,33,155,other,nonsmoker,0,0,0,3,2551
2,87,0,20,105,white,smoker,0,0,0,1,2557
3,88,0,21,108,white,smoker,0,0,1,2,2594
4,89,0,18,107,white,smoker,0,0,1,0,2600


尤度関数の最適化をscipyの関数を使って行い、結果を出力  
引数として、group(stataのi.と同じでダミー変数を作る項を指定、defaultだと数値はそのまま扱い、文字列はダミー変数処理とかでいいと思う)  
ダミー変数で基準となる項目の指定とかも引数にできたらいいかも

おそらく一番数が多いやつがdefaultになっている。

In [9]:
data.dtypes

id          int16
low          int8
age          int8
lwt         int16
race     category
smoke    category
ptl          int8
ht           int8
ui           int8
ftv          int8
bwt         int16
dtype: object

In [10]:
category_variables = data.dtypes[(data.dtypes == 'category') | (data.dtypes == 'object')].index
category_mode = {}
for variable in category_variables:
    category_mode[variable] = data[variable].value_counts().index[0]
print(category_mode)

{'race': 'white', 'smoke': 'nonsmoker'}


In [11]:
def get_category_modes(df):
    """
    カテゴリータイプまたはオブジェクトタイプの列の最頻値を返す関数
    """
    category_variables = df.dtypes[(df.dtypes == 'category') | (df.dtypes == 'object')].index
    category_mode = {}
    for variable in category_variables:
        category_mode[variable] = df[variable].value_counts().index[0]
    return category_mode

In [12]:
X_dummy = pd.get_dummies(X)
category_modes = get_category_modes(X)
for index, value in category_modes.items():
    X_dummy.drop(index + '_' + value, axis=1, inplace=True)

In [13]:
len(X_dummy.iloc[0])

11

In [14]:
X_dummy.iloc[0]

id                85
age               19
lwt              182
ptl                0
ht                 0
ui                 1
ftv                0
bwt             2523
race_black         1
race_other         0
smoke_smoker       0
Name: 0, dtype: int16

In [15]:
X_dummy.iloc[0]*np.array([3, 4, 0, 1, 2, 2, 2, 2, 2, 2, 2])

id               255
age               76
lwt                0
ptl                0
ht                 0
ui                 2
ftv                0
bwt             5046
race_black         2
race_other         0
smoke_smoker       0
Name: 0, dtype: int64

In [18]:
np.dot(X_dummy.iloc[0], np.array([3, 4, 0, 1, 2, 2, 2, 2, 2, 2, 2]))

5381

In [16]:
import functools
import math
from scipy import optimize

In [106]:
def logistic_ln_likelyhood(b, X, y):
    """
    bはdummy化したXの列と同じ長さのnp.arrayを想定
    """
    ln_likelyhood = 0
    for X_i, y_i in zip(X.itertuples(name=None), y):
        X_i = X_i[1:]
        hat_y = 1/(1+math.exp(-np.dot(X_i, b)))
        if y_i == 1 and hat_y == 1:
            continue
        if y_i == 0 and hat_y == 0:
            continue
        ln_likelyhood += ((1-y_i)*math.log(1-hat_y)+y_i*math.log(hat_y))
    print(-ln_likelyhood)
    return - ln_likelyhood

def ln_likelyhood_func(X, y):
    return functools.partial(logistic_ln_likelyhood, X=X, y=y)

In [107]:
X_dummy_ = X_dummy.copy()
bwt = X_dummy['bwt']
X_dummy_['bwt'] = (bwt-bwt.mean()) / bwt.std()
lwt = X_dummy['lwt']
X_dummy_['lwt'] = (lwt-lwt.mean()) / lwt.std()
age = X_dummy['age']
X_dummy_['age'] = (age-age.mean()) / age.std()
X_dummy_['_cons'] = 1

In [108]:
func = ln_likelyhood_func(X_dummy_.drop('id', axis=1).drop('bwt', axis=1).drop('ftv', axis=1), y)

In [109]:
b = np.array([1, 1.1, 1, 1, 1, 1, 1.2, 1, 0.5])
func(b)

293.93539932446276


293.93539932446276

In [110]:
optimize.minimize(func, b)

293.93539932446276
293.9353998257139
293.935399817653
293.93539954308994
293.93539938501164
293.9353994770132
293.93539947708985
293.93539972756264
293.9353997808308
293.93540062989484
293.93539932446276
194.84325943385508
194.84325943385508
194.8432599445404
194.84325998422335
194.84325960548355
194.84325948053313
194.8432595418534
194.84325955053274
194.84325970112764
194.84325974037966
194.8432603502341
184.72357705277213
184.72357705277213
184.72357656465965
184.7235765529261
184.72357715918253
184.72357703658864
184.7235771634444
184.72357708948394
184.72357740634126
184.72357728290805
184.72357773959104
167.79372924282995
167.79372924282995
167.79372887859478
167.79372870234783
167.7937293244905
167.79372921948183
167.7937293366645
167.793729267441
167.79372953178495
167.79372947944503
167.79372985580105
140.5792197629951
140.5792197629951
140.5792193448409
140.57921937177795
140.5792197514211
140.57921974575083
140.57921980621668
140.57921978728524
140.57921992053468
140.5792199

      fun: 100.7239955662642
 hess_inv: array([[ 0.03886682, -0.00471066, -0.00894792,  0.00548942,  0.00953937,
         0.01603309,  0.01056777,  0.00334815, -0.00500226],
       [-0.00471066,  0.04526909,  0.00503468, -0.04725472,  0.0055532 ,
        -0.02222248,  0.01396139,  0.00468161,  0.00279362],
       [-0.00894792,  0.00503468,  0.12701235, -0.00152512, -0.03068614,
        -0.00460929, -0.00629991, -0.02011598, -0.01182899],
       [ 0.00548942, -0.04725472, -0.00152512,  0.48185499,  0.02965175,
         0.0112747 , -0.01189682, -0.00864535, -0.03992437],
       [ 0.00953937,  0.0055532 , -0.03068614,  0.02965175,  0.21654758,
         0.01436691, -0.00492699, -0.00384094, -0.02937264],
       [ 0.01603309, -0.02222248, -0.00460929,  0.0112747 ,  0.01436691,
         0.28545971,  0.08482846,  0.03977724, -0.10306839],
       [ 0.01056777,  0.01396139, -0.00629991, -0.01189682, -0.00492699,
         0.08482846,  0.20696521,  0.08934061, -0.13133874],
       [ 0.00334815,  

In [96]:
X_dummy_.head().drop('id', axis=1)

,age,lwt,ptl,ht,ui,ftv,bwt,race_black,race_other,smoke_smoker,_cons
0,-0.799840,1.706611,0,0,1,0,-0.577883,1,0,0,1
1,1.842328,0.823541,0,0,0,3,-0.539475,0,1,0,1
2,-0.611114,-0.811774,0,0,0,1,-0.531244,0,0,1,1
3,-0.422387,-0.713655,0,0,1,2,-0.480491,0,0,1,1
4,-0.988566,-0.746361,0,0,1,0,-0.472261,0,0,1,1


In [97]:
X_dummy_.drop('id', axis=1).drop('bwt', axis=1).drop('ftv', axis=1).head(1)

,age,lwt,ptl,ht,ui,race_black,race_other,smoke_smoker,_cons
0,-0.79984,1.706611,0,0,1,1,0,0,1


In [98]:
a = X_dummy_.drop('id', axis=1).drop('bwt', axis=1).drop('ftv', axis=1).head(1)